Use opencv for cropping, simple function

In [18]:
import cv2, random
import numpy as np

In [19]:
def autoCrop(input_path, output_path="segment/{}.jpg", option="cropped", gaussian_size=9, canny_min=0, canny_max=100, dilation_size=25, approx_proportion=0.25, height_thresh_proportion=0.1, width_thresh_proportion=0.1, resize_factor=1):
    img = cv2.imread(input_path,1)
    img_1 = cv2.cv2.copyMakeBorder(img,0,0,0,0,cv2.BORDER_REPLICATE)
    img_2 = cv2.cv2.copyMakeBorder(img,0,0,0,0,cv2.BORDER_REPLICATE)
    
    # Gussian Blur
    blur = cv2.GaussianBlur(img, (gaussian_size, gaussian_size), 0)
    # Canny Edge Detection
    edged = cv2.Canny(blur, canny_min, canny_max)
    # dialation
    dilated = cv2.dilate(edged, np.ones((dilation_size, dilation_size)))
    # find contours
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # draw contour
    img_1 = cv2.drawContours(img_1, contours, -1, (0,255,0), 3) # in green
    for i, contour in enumerate(contours):
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, approx_proportion*perimeter, True)
        if (len(approx)!=4): # rule out not rectangles
            rect = cv2.boundingRect(contour)
            x, y, w, h = [r*resize_factor for r in rect]
            if (w>width_thresh_proportion*img_2.shape[1] and h>height_thresh_proportion*img_2.shape[0]):
                cv2.rectangle(img_2, (x,y), ((x+w), (y+h)), (0, 0, 255), 10) # in red
                crop_img = img[y:y+h, x:x+w]
                cv2.imwrite(output_path.format(i), crop_img)

In [20]:
autoCrop('pic/21-43-11/56b0183fdbff00aed3f9d0e369184128-1.png')